# Q1

## Q1.1

In [1]:
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.ml import Pipeline,Transformer
from pyspark.ml.feature import Imputer,StandardScaler,StringIndexer,OneHotEncoder, VectorAssembler

from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np

col_names = ["duration","protocol_type","service","flag","src_bytes",
"dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
"logged_in","num_compromised","root_shell","su_attempted","num_root",
"num_file_creations","num_shells","num_access_files","num_outbound_cmds",
"is_host_login","is_guest_login","count","srv_count","serror_rate",
"srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
"diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
"dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
"dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
"dst_host_rerror_rate","dst_host_srv_rerror_rate","class","difficulty"]

nominal_cols = ['protocol_type','service','flag']
binary_cols = ['land', 'logged_in', 'root_shell', 'su_attempted', 'is_host_login',
'is_guest_login']
continuous_cols = ['duration' ,'src_bytes', 'dst_bytes', 'wrong_fragment' ,'urgent', 'hot',
'num_failed_logins', 'num_compromised', 'num_root' ,'num_file_creations',
'num_shells', 'num_access_files', 'num_outbound_cmds', 'count' ,'srv_count',
'serror_rate', 'srv_serror_rate' ,'rerror_rate' ,'srv_rerror_rate',
'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate' ,'dst_host_count',
'dst_host_srv_count' ,'dst_host_same_srv_rate' ,'dst_host_diff_srv_rate',
'dst_host_same_src_port_rate' ,'dst_host_srv_diff_host_rate',
'dst_host_serror_rate' ,'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
'dst_host_srv_rerror_rate']


DoS=['apache2','back','land','neptune','mailbomb','pod','processtable','smurf','teardrop','udpstorm','worm']
Probe=['ipsweep','mscan','nmap','portsweep','saint','satan']
U2R = ['buffer_overflow','loadmodule','perl','ps','rootkit','sqlattack','xterm']
R2L = ['ftp_write','guess_passwd','httptunnel','imap','multihop','named','phf','sendmail','Snmpgetattack','spy',
      'snmpguess','warezclient','warezmaster','xlock','xsnoop']


class OutcomeCreater(Transformer): # this defines a transformer that creates the outcome column
    
    def __init__(self):
        super().__init__()

    def _transform(self, dataset):
        label_to_binary = udf(lambda name: 0.0 if name == 'normal' else 1.0 if name in DoS else 2.0 if name in Probe
                             else 3.0 if name in U2R else 4.0)
        output_df = dataset.withColumn('outcome', label_to_binary(col('class'))).drop("class")  
        output_df = output_df.withColumn('outcome', col('outcome').cast(DoubleType()))
        output_df = output_df.drop('difficulty')
        return output_df

class FeatureTypeCaster(Transformer): # this transformer will cast the columns as appropriate types  
    def __init__(self):
        super().__init__()

    def _transform(self, dataset):
        output_df = dataset
        for col_name in binary_cols + continuous_cols:
            output_df = output_df.withColumn(col_name,col(col_name).cast(DoubleType()))

        return output_df
class ColumnDropper(Transformer): # this transformer drops unnecessary columns
    def __init__(self, columns_to_drop = None):
        super().__init__()
        self.columns_to_drop=columns_to_drop
    def _transform(self, dataset):
        output_df = dataset
        for col_name in self.columns_to_drop:
            output_df = output_df.drop(col_name)
        return output_df

def get_preprocess_pipeline():
    # Stage where columns are casted as appropriate types
    stage_typecaster = FeatureTypeCaster()

    # Stage where nominal columns are transformed to index columns using StringIndexer
    nominal_id_cols = [x+"_index" for x in nominal_cols]
    nominal_onehot_cols = [x+"_encoded" for x in nominal_cols]
    stage_nominal_indexer = StringIndexer(inputCols = nominal_cols, outputCols = nominal_id_cols )

    # Stage where the index columns are further transformed using OneHotEncoder
    stage_nominal_onehot_encoder = OneHotEncoder(inputCols=nominal_id_cols, outputCols=nominal_onehot_cols)

    # Stage where all relevant features are assembled into a vector (and dropping a few)
    feature_cols = continuous_cols+binary_cols+nominal_onehot_cols
    corelated_cols_to_remove = ["dst_host_serror_rate","srv_serror_rate","dst_host_srv_serror_rate",
                     "srv_rerror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate"]
    for col_name in corelated_cols_to_remove:
        feature_cols.remove(col_name)
    stage_vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="vectorized_features")

    # Stage where we scale the columns
    stage_scaler = StandardScaler(inputCol= 'vectorized_features', outputCol= 'features')
    

    # Stage for creating the outcome column representing whether there is attack 
    stage_outcome = OutcomeCreater()

    # Removing all unnecessary columbs, only keeping the 'features' and 'outcome' columns
    stage_column_dropper = ColumnDropper(columns_to_drop = nominal_cols+nominal_id_cols+
        nominal_onehot_cols+ binary_cols + continuous_cols + ['vectorized_features'])
    # Connect the columns into a pipeline
    pipeline = Pipeline(stages=[stage_typecaster,stage_nominal_indexer,stage_nominal_onehot_encoder,
        stage_vector_assembler,stage_scaler,stage_outcome,stage_column_dropper])
    return pipeline 

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("GenericAppName") \
    .getOrCreate()

nslkdd_raw = spark.read.csv('Data/KDDTrain+.txt',header=False).toDF(*col_names)
nslkdd_test_raw = spark.read.csv('Data/KDDTest+.txt',header=False).toDF(*col_names)
preprocess_pipeline = get_preprocess_pipeline()
preprocess_pipeline_model = preprocess_pipeline.fit(nslkdd_raw)

nslkdd_df = preprocess_pipeline_model.transform(nslkdd_raw)
nslkdd_df_test = preprocess_pipeline_model.transform(nslkdd_test_raw)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/07 14:03:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


22/11/07 14:03:16 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [3]:
nslkdd_df.show()

+--------------------+-------+
|            features|outcome|
+--------------------+-------+
|(113,[1,13,14,17,...|    0.0|
|(113,[1,13,14,17,...|    0.0|
|(113,[13,14,15,17...|    1.0|
|(113,[1,2,13,14,1...|    0.0|
|(113,[1,2,13,14,1...|    0.0|
|(113,[13,14,16,17...|    1.0|
|(113,[13,14,15,17...|    1.0|
|(113,[13,14,15,17...|    1.0|
|(113,[13,14,15,17...|    1.0|
|(113,[13,14,15,17...|    1.0|
|(113,[13,14,16,17...|    1.0|
|(113,[13,14,15,17...|    1.0|
|(113,[1,2,13,14,1...|    0.0|
|(113,[1,13,14,17,...|    4.0|
|(113,[13,14,15,18...|    1.0|
|(113,[13,14,15,17...|    1.0|
|(113,[1,2,13,14,1...|    0.0|
|(113,[1,13,14,17,...|    2.0|
|(113,[1,2,13,14,1...|    0.0|
|(113,[1,2,13,14,1...|    0.0|
+--------------------+-------+
only showing top 20 rows



## Q1.2

In [4]:
to_array = udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))

nslkdd_df_train = nslkdd_df 
nslkdd_df_validate,nslkdd_df_test = nslkdd_df_test.randomSplit([0.5,0.5])

nslkdd_df_train_pandas = nslkdd_df_train.withColumn('features', to_array('features')).toPandas()
nslkdd_df_validate_pandas = nslkdd_df_validate.withColumn('features', to_array('features')).toPandas()
nslkdd_df_test_pandas = nslkdd_df_test.withColumn('features', to_array('features')).toPandas()

In [5]:
import tensorflow as tf
from tensorflow import keras 

# Converting the pandas DataFrame to tensors
# Note we are using 3 data sets train, validate, test

x_train = tf.constant(np.array(nslkdd_df_train_pandas['features'].values.tolist()))
y_train = tf.constant(np.array(nslkdd_df_train_pandas['outcome'].values.tolist()))

x_validate = tf.constant(np.array(nslkdd_df_validate_pandas['features'].values.tolist()))
y_validate = tf.constant(np.array(nslkdd_df_validate_pandas['outcome'].values.tolist()))

x_test = tf.constant(np.array(nslkdd_df_test_pandas['features'].values.tolist()))
y_test = tf.constant(np.array(nslkdd_df_test_pandas['outcome'].values.tolist()))

2022-11-07 14:03:37.981005: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
model_multiclass = keras.Sequential( [keras.layers.Dense(10,activation='relu'),
                           keras.layers.Dense(10,activation='relu'),
                           keras.layers.Dense(10,activation='relu'),
                           keras.layers.Dense(10,activation='relu') ,
                           keras.layers.Dense(5)] )

model_multiclass.compile(optimizer = 'sgd',
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()])

model_multiclass.fit(x_train,y_train, epochs = 5,validation_data=(x_validate,y_validate),verbose = 2)

Epoch 1/5
3937/3937 - 4s - loss: 0.2219 - sparse_categorical_accuracy: 0.9356 - val_loss: 1.4200 - val_sparse_categorical_accuracy: 0.7448 - 4s/epoch - 981us/step
Epoch 2/5
3937/3937 - 3s - loss: 0.0748 - sparse_categorical_accuracy: 0.9766 - val_loss: 1.6924 - val_sparse_categorical_accuracy: 0.7398 - 3s/epoch - 823us/step
Epoch 3/5
3937/3937 - 3s - loss: 0.0509 - sparse_categorical_accuracy: 0.9834 - val_loss: 1.9222 - val_sparse_categorical_accuracy: 0.7662 - 3s/epoch - 870us/step
Epoch 4/5
3937/3937 - 3s - loss: 0.0429 - sparse_categorical_accuracy: 0.9849 - val_loss: 2.0552 - val_sparse_categorical_accuracy: 0.7667 - 3s/epoch - 849us/step
Epoch 5/5
3937/3937 - 3s - loss: 0.0373 - sparse_categorical_accuracy: 0.9877 - val_loss: 2.0741 - val_sparse_categorical_accuracy: 0.7601 - 3s/epoch - 857us/step


In [7]:
model_multiclass.evaluate(x_test, y_test, verbose=2)

351/351 - 0s - loss: 2.0598 - sparse_categorical_accuracy: 0.7570 - 260ms/epoch - 742us/step


[2.0597593784332275, 0.7569618225097656]

## Q1.3

In [12]:
import datetime
log_dir = "logs14813/myfirstlog/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model_multiclass.fit(x=x_train, y=y_train, 
          epochs=20, verbose = 2,
          validation_data=(x_validate, y_validate), 
          callbacks=[tensorboard_callback])

Epoch 1/20
3937/3937 - 4s - loss: 0.0207 - sparse_categorical_accuracy: 0.9936 - val_loss: 2.9599 - val_sparse_categorical_accuracy: 0.7707 - 4s/epoch - 906us/step
Epoch 2/20
3937/3937 - 3s - loss: 0.0203 - sparse_categorical_accuracy: 0.9936 - val_loss: 3.1242 - val_sparse_categorical_accuracy: 0.7607 - 3s/epoch - 864us/step
Epoch 3/20
3937/3937 - 3s - loss: 0.0200 - sparse_categorical_accuracy: 0.9937 - val_loss: 3.0215 - val_sparse_categorical_accuracy: 0.7609 - 3s/epoch - 864us/step
Epoch 4/20
3937/3937 - 4s - loss: 0.0193 - sparse_categorical_accuracy: 0.9939 - val_loss: 3.1656 - val_sparse_categorical_accuracy: 0.7491 - 4s/epoch - 900us/step
Epoch 5/20
3937/3937 - 4s - loss: 0.0190 - sparse_categorical_accuracy: 0.9941 - val_loss: 3.2726 - val_sparse_categorical_accuracy: 0.7544 - 4s/epoch - 927us/step
Epoch 6/20
3937/3937 - 3s - loss: 0.0190 - sparse_categorical_accuracy: 0.9942 - val_loss: 3.1359 - val_sparse_categorical_accuracy: 0.7628 - 3s/epoch - 875us/step
Epoch 7/20
3937/

In [9]:
import os
os.environ['TENSORBOARD_BINARY'] = "logs14813/myfirstlog/"

In [11]:
%load_ext tensorboard
%tensorboard --logdir logs14813/myfirstlog/

Reusing TensorBoard on port 6006 (pid 5880), started 0:05:30 ago. (Use '!kill 5880' to kill it.)

# Q2

## Q2.1

In [13]:
to_array = udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))

nslkdd_df_train = nslkdd_df 
nslkdd_df_validate,nslkdd_df_test = nslkdd_df_test.randomSplit([0.5,0.5])

In [14]:
train = nslkdd_df_train.union(nslkdd_df_validate)
train.count() 

131695

In [15]:
import pyspark.sql.functions as F
train = train.orderBy(F.rand())

In [16]:
import pandas as pd

In [17]:
def crossvalidator_k_equal_6(hparams,logdir,train):
    
    # model
    model = keras.Sequential()
    for _ in range(hparams[HP_DEPTH]):
        model.add(keras.layers.Dense(hparams[HP_WIDTH],activation='relu'))
    model.add(keras.layers.Dense(5))
    
    model.compile(optimizer = 'sgd',
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[keras.metrics.SparseCategoricalAccuracy()])
    # data
    #test = test.withColumn('features', to_array('features')).toPandas()
    #x_test = tf.constant(np.array(nslkdd_df_test_pandas['features'].values.tolist()))
    #y_test = tf.constant(np.array(nslkdd_df_test_pandas['outcome'].values.tolist()))
    
    cv1,cv2,cv3,cv4,cv5,cv6 = train.randomSplit([1/6, 1/6, 1/6, 1/6, 1/6, 1/6])
    cv1_pandas = cv1.withColumn('features', to_array('features')).toPandas()
    cv2_pandas = cv2.withColumn('features', to_array('features')).toPandas()
    cv3_pandas = cv3.withColumn('features', to_array('features')).toPandas()
    cv4_pandas = cv4.withColumn('features', to_array('features')).toPandas()
    cv5_pandas = cv5.withColumn('features', to_array('features')).toPandas()
    cv6_pandas = cv6.withColumn('features', to_array('features')).toPandas()
    
    
    # for loop 
    cv = [cv1_pandas, cv2_pandas, cv3_pandas, cv4_pandas, cv5_pandas, cv6_pandas]
    acc = []
    n = 0
    for i in cv:
        #print('check1')
        cvv = cv.copy()
        #print('check2')
        x_validate = tf.constant(np.array(i['features'].values.tolist()))
        y_validate = tf.constant(np.array(i['outcome'].values.tolist()))
        #print('check3')
        del cvv[n]
        n +=1
        train_data = pd.concat(cvv,axis=0,join='outer')
        #print('check4')
        x_train = tf.constant(np.array(train_data['features'].values.tolist()))
        y_train = tf.constant(np.array(train_data['outcome'].values.tolist()))

        #print('check5')
        history = model.fit(x_train, y_train, epochs=5, verbose = 2,
                            callbacks=[tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)],
                            validation_data = (x_validate, y_validate))
        #print('check6')
        a = np.max(history.history["val_sparse_categorical_accuracy"])
        #print('a:',a)        
        acc.append(a)
    acc = np.array(acc)
    return acc.mean()
    


In [19]:
from tensorboard.plugins.hparams import api as hp

HP_WIDTH = hp.HParam('NN_width', hp.Discrete([20]))
HP_DEPTH = hp.HParam('NN_depth', hp.Discrete([4]))

with tf.summary.create_file_writer('logs14813/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_WIDTH, HP_DEPTH],
        metrics=[hp.Metric("AUC"),hp.Metric('Accuracy')],
      )

In [20]:
hparams = {
        HP_WIDTH: 20,
        HP_DEPTH: 4,
    }

accuracy = crossvalidator_k_equal_6(hparams,log_dir,train)

Epoch 1/5
3429/3429 - 4s - loss: 0.2289 - sparse_categorical_accuracy: 0.9368 - val_loss: 0.1171 - val_sparse_categorical_accuracy: 0.9657 - 4s/epoch - 1ms/step
Epoch 2/5
3429/3429 - 3s - loss: 0.0921 - sparse_categorical_accuracy: 0.9706 - val_loss: 0.0834 - val_sparse_categorical_accuracy: 0.9734 - 3s/epoch - 954us/step
Epoch 3/5
3429/3429 - 3s - loss: 0.0681 - sparse_categorical_accuracy: 0.9794 - val_loss: 0.0876 - val_sparse_categorical_accuracy: 0.9724 - 3s/epoch - 984us/step
Epoch 4/5
3429/3429 - 3s - loss: 0.0576 - sparse_categorical_accuracy: 0.9837 - val_loss: 0.0905 - val_sparse_categorical_accuracy: 0.9809 - 3s/epoch - 950us/step
Epoch 5/5
3429/3429 - 4s - loss: 0.0532 - sparse_categorical_accuracy: 0.9850 - val_loss: 0.0549 - val_sparse_categorical_accuracy: 0.9836 - 4s/epoch - 1ms/step
Epoch 1/5
3424/3424 - 4s - loss: 0.0504 - sparse_categorical_accuracy: 0.9856 - val_loss: 0.0456 - val_sparse_categorical_accuracy: 0.9855 - 4s/epoch - 1ms/step
Epoch 2/5
3424/3424 - 4s - l

In [23]:
print(accuracy)

0.9892603754997253


## Q2.2

In [34]:
from tensorboard.plugins.hparams import api as hp

HP_WIDTH = hp.HParam('NN_width', hp.Discrete([20,30]))
HP_DEPTH = hp.HParam('NN_depth', hp.Discrete([4,6]))

with tf.summary.create_file_writer('logs14813/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_WIDTH, HP_DEPTH],
        metrics=[hp.Metric('Accuracy')],
      )


In [35]:
accuracy_list = []
for hp_width in HP_WIDTH.domain.values:
    for hp_depth in (HP_DEPTH.domain.values):
        hparams = {
            HP_WIDTH: hp_width,
            HP_DEPTH: hp_depth,
        }
        run_name = f"run-WIDTH{int(hparams[HP_WIDTH])}-DEPTH{hparams[HP_DEPTH]}"
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})

        run_dir = 'logs14813/hparam_tuning/' + run_name
        accuracy = crossvalidator_k_equal_6(hparams,run_dir,train)
        accuracy_list.append(accuracy)

        with tf.summary.create_file_writer(run_dir).as_default():
            hp.hparams(hparams)  # record the values used in this trial
            tf.summary.scalar("Accuracy", accuracy, step=1)

--- Starting trial: run-WIDTH20-DEPTH4
{'NN_width': 20, 'NN_depth': 4}


Epoch 1/5
3437/3437 - 4s - loss: 0.2054 - sparse_categorical_accuracy: 0.9390 - val_loss: 0.0944 - val_sparse_categorical_accuracy: 0.9680 - 4s/epoch - 1ms/step
Epoch 2/5
3437/3437 - 4s - loss: 0.0800 - sparse_categorical_accuracy: 0.9755 - val_loss: 0.0654 - val_sparse_categorical_accuracy: 0.9786 - 4s/epoch - 1ms/step
Epoch 3/5
3437/3437 - 4s - loss: 0.0631 - sparse_categorical_accuracy: 0.9816 - val_loss: 0.0576 - val_sparse_categorical_accuracy: 0.9815 - 4s/epoch - 1ms/step
Epoch 4/5
3437/3437 - 4s - loss: 0.0552 - sparse_categorical_accuracy: 0.9834 - val_loss: 0.0527 - val_sparse_categorical_accuracy: 0.9835 - 4s/epoch - 1ms/step
Epoch 5/5
3437/3437 - 4s - loss: 0.0506 - sparse_categorical_accuracy: 0.9847 - val_loss: 0.0481 - val_sparse_categorical_accuracy: 0.9862 - 4s/epoch - 1ms/step
Epoch 1/5
3431/3431 - 4s - loss: 0.0480 - sparse_categorical_accuracy: 0.9858 - val_loss: 0.0464 - val_sparse_categorical_accuracy: 0.9844 - 4s/epoch - 1ms/step
Epoch 2/5
3431/3431 - 4s - loss: 0

Epoch 1/5
3426/3426 - 4s - loss: 0.2940 - sparse_categorical_accuracy: 0.9183 - val_loss: 0.1424 - val_sparse_categorical_accuracy: 0.9636 - 4s/epoch - 1ms/step
Epoch 2/5
3426/3426 - 4s - loss: 0.1079 - sparse_categorical_accuracy: 0.9686 - val_loss: 0.0751 - val_sparse_categorical_accuracy: 0.9756 - 4s/epoch - 1ms/step
Epoch 3/5
3426/3426 - 4s - loss: 0.0705 - sparse_categorical_accuracy: 0.9776 - val_loss: 0.0653 - val_sparse_categorical_accuracy: 0.9820 - 4s/epoch - 1ms/step
Epoch 4/5
3426/3426 - 4s - loss: 0.0586 - sparse_categorical_accuracy: 0.9820 - val_loss: 0.0656 - val_sparse_categorical_accuracy: 0.9793 - 4s/epoch - 1ms/step
Epoch 5/5
3426/3426 - 4s - loss: 0.0524 - sparse_categorical_accuracy: 0.9848 - val_loss: 0.0497 - val_sparse_categorical_accuracy: 0.9870 - 4s/epoch - 1ms/step
Epoch 1/5
3430/3430 - 5s - loss: 0.0486 - sparse_categorical_accuracy: 0.9859 - val_loss: 0.0429 - val_sparse_categorical_accuracy: 0.9871 - 5s/epoch - 1ms/step
Epoch 2/5
3430/3430 - 6s - loss: 0

Epoch 1/5
3433/3433 - 4s - loss: 0.2006 - sparse_categorical_accuracy: 0.9437 - val_loss: 0.0996 - val_sparse_categorical_accuracy: 0.9689 - 4s/epoch - 1ms/step
Epoch 2/5
3433/3433 - 3s - loss: 0.0776 - sparse_categorical_accuracy: 0.9734 - val_loss: 0.0672 - val_sparse_categorical_accuracy: 0.9825 - 3s/epoch - 1ms/step
Epoch 3/5
3433/3433 - 3s - loss: 0.0579 - sparse_categorical_accuracy: 0.9832 - val_loss: 0.0597 - val_sparse_categorical_accuracy: 0.9834 - 3s/epoch - 966us/step
Epoch 4/5
3433/3433 - 4s - loss: 0.0497 - sparse_categorical_accuracy: 0.9851 - val_loss: 0.0530 - val_sparse_categorical_accuracy: 0.9860 - 4s/epoch - 1ms/step
Epoch 5/5
3433/3433 - 4s - loss: 0.0455 - sparse_categorical_accuracy: 0.9864 - val_loss: 0.0506 - val_sparse_categorical_accuracy: 0.9847 - 4s/epoch - 1ms/step
Epoch 1/5
3424/3424 - 4s - loss: 0.0425 - sparse_categorical_accuracy: 0.9872 - val_loss: 0.0435 - val_sparse_categorical_accuracy: 0.9861 - 4s/epoch - 1ms/step
Epoch 2/5
3424/3424 - 4s - loss:

Epoch 1/5
3425/3425 - 4s - loss: 0.1988 - sparse_categorical_accuracy: 0.9400 - val_loss: 0.0811 - val_sparse_categorical_accuracy: 0.9726 - 4s/epoch - 1ms/step
Epoch 2/5
3425/3425 - 4s - loss: 0.0667 - sparse_categorical_accuracy: 0.9806 - val_loss: 0.0636 - val_sparse_categorical_accuracy: 0.9791 - 4s/epoch - 1ms/step
Epoch 3/5
3425/3425 - 4s - loss: 0.0528 - sparse_categorical_accuracy: 0.9846 - val_loss: 0.0486 - val_sparse_categorical_accuracy: 0.9863 - 4s/epoch - 1ms/step
Epoch 4/5
3425/3425 - 4s - loss: 0.0459 - sparse_categorical_accuracy: 0.9860 - val_loss: 0.0461 - val_sparse_categorical_accuracy: 0.9870 - 4s/epoch - 1ms/step
Epoch 5/5
3425/3425 - 4s - loss: 0.0422 - sparse_categorical_accuracy: 0.9868 - val_loss: 0.0397 - val_sparse_categorical_accuracy: 0.9888 - 4s/epoch - 1ms/step
Epoch 1/5
3434/3434 - 4s - loss: 0.0404 - sparse_categorical_accuracy: 0.9874 - val_loss: 0.0384 - val_sparse_categorical_accuracy: 0.9882 - 4s/epoch - 1ms/step
Epoch 2/5
3434/3434 - 4s - loss: 0

In [36]:
%tensorboard --logdir logs14813/hparam_tuning/

Reusing TensorBoard on port 6007 (pid 6599), started 0:39:43 ago. (Use '!kill 6599' to kill it.)

## Q3.3

In [37]:
print(accuracy_list)

[0.9898701012134552, 0.9900753696759542, 0.9904125531514486, 0.9912060697873434]
